# 2. Data acquisition and cleaning  
## 2.1 Data sources  
The data is originally from Seattle Department of Transportation (SDOT), which covers the collision happened in Seattle from 2004-01-01 to 2020-05-20, each case is labeled with a severity code, and contains a variety of attributes of the case. 
There is also a explaination document "Metadata" for the meaning of each column.  
## 2.2 Data cleaning  & Feature selection
By checking with missing data in the list, we can find there are several columns missing a large number of values. But before removing any rows or columns, it's better to understand the meaning of each column. Since if one column is not suitable or necessary to be kept as a feature, then any missing value in that column doesn't matter at all.  
  
So that we can start by reading the file "Metadata" to rule out unnecessary columns first, then deal with missing values in the remaining columns. It’s easy to find out that a couple of the columns are duplicated peers such as “SEVERITYCODE” and “SEVERITYDESC”, they are actually equal to each other; or some are identifications of a case, such as “OBJECTID”, “INCKEY” and so on, those columns should just be excluded from the data set.  
  
And there are three columns which are quite interesting, "INJURIES", "SERIOUSINJURIES" and "FATALITIES". After a quick analysis, it's easy to find that there is a strong logic between these columns and the label "SEVERITYCODE". Where the one-hot logics are like this: code-1=000, code-2=100, code-2b=x10, code-3=xx1 (code-0 shall be dropped). But a couple of exceptions of code-2b are found having the combination of “x00”, while I also noticed that for these samples, it says “Unmatched” in column “STATUS”. So that, in conclusion, it should be reasonable to drop all these columns, since they play like dummies to the labels. And all rows marked as “Unmatched” shall also be dropped before column drops.
***
| Column to be dropped          | Reason to drop                  |
| :-----                        | :---------------                |
| X                             | not explained in Metadata       |
| Y                             | not explained in Metadata       |
| OBJECTID                      | case id, no meaning             |
| INCKEY                        | case id, no meaning             |
| COLDETKEY                        | case id, no meaning             |
| REPORTNO                      | case id, no meaning             |
| STATUS                        | drop "Unmatched" rows first, then drop the column       |
| INTKEY                        | a combination of other columns  |
| LOCATION                      | too detailed address info       |
| EXCEPTRSNCODE                 | not explained in Metadata       |
| EXCEPTRSNDESC                 | not explained in Metadata       |
| SEVERITYDESC                  | duplicated column               |
| INCDATE                  | duplicated column               |
| SDOT_COLDESC                  | duplicated column               |
| SDOTCOLNUM                    | case id, no meaning             |
| ST_COLDESC                    | duplicated column               |
| SEGLANEKEY                    | too many missing values, and meaning is unclear         |
| CROSSWALKKEY                  | too many missing values, and meaning is unclear         |
| INJURIES                  | dummy column        | 
| SERIOUSINJURIES                  | dummy column         | 
| FATALITIES                  | dummy column         | 
  

For a better understanding of missing data of remaining columns, replace empty value with "N" in column "INATTENTIONIND", "PEDROWNOTGRNT" and "SPEEDING" as only "Y" is marked out in the columns. In column "UNDERINFL" there are both "Y/N" and "1/0", for standardization, replace "Y/N" into "1/0" respectively. Now we can drop entitle rows with missing value in any column. 
  
The next step of data cleaning is to do some conversion of certain values for standardization: format "INCDTTM" into datetime and rename as "DATETIME", replace "Y/N" into "1/0" in column "INATTENTIONIND", "PEDROWNOTGRNT" and "SPEEDING".  
  

## 2.3 Data balancing  
There are 4 labels (categories) in the dataset (1,2,2b,3), but the number of samples within each category are not balanced as the statistics shows:  
***
1--------127280  
2--------57162  
2b-------3014  
3--------337   
  
If all categories are to be used as individual label with balanced samples, then the result will be around 300 from each category and 1200 in total. But if comparing to the size of the original data, that’s quite a waste. Instead, we can consider to bin them into two categories as ”Injured” and “Not Injured” for reasons:
1.	Although 2, 2b, 3 have different meanings, but they all can be considered as “human injury”, which in real life always mean involvement of more public departments; while if there is no injury, most of the cases will be much easier to handle.
2.	By converting into a binary classification problem, more classifier models can be applied, other than being limited with logistic regression.
  
While the original list contains data ranging from year 2004 to 2020, it might not be necessary to keep all of them. Since the traffic condition and vehicle technology have been changed quite a lot, data from a long time ago may not be able to correctly reflect current situations, and plus that there are enough samples. We can only keep the latest at around 20% of total samples, sort the data by “DATETIME” in descending order and then keep the top 30000 rows.  
